In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sb

In [2]:
ceo1999 = pd.read_csv('telco.txt', sep='\t')
ceo1999.head()

,tenure,age,marital,address,income,ed,employ,retire,gender,longmon,wiremon,churn
1,13,44,Married,9,64,College degree,5,No,Male,3.70,0.0,Yes
2,11,33,Married,7,136,Post-undergraduate degree,5,No,Male,4.40,35.7,Yes
3,68,52,Married,24,116,Did not complete high school,29,No,Female,18.15,0.0,No
4,33,33,Unmarried,12,33,High school degree,0,No,Female,9.45,0.0,Yes
5,23,30,Married,9,30,Did not complete high school,2,No,Male,6.30,0.0,No


# Problem 4

## Task 1
Have a closer look at the deﬁnitions of the variables and analyze which of them might require a separate treatment. Consider for example the variable ed. There are two possibilities how the variable ed can be included into the model (one with dummy variables, the other one without dummies). Think about these two approaches and suggest which approach is more appropriate. Motivate your decision.

In given dataset the are multiple categorical variables such as:
- marital with values Married and Unmarried;
- retire with Yes and No values;
- gender with Male and Female values;
- churn with values Yes and No;
- ed with 5 distinct values - Did not complete high school, High school degree, Some college, College degree, Post-undergraduate degree.